In [ ]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install fastapi nest-asyncio pyngrok uvicorn

In [ ]:
if True:
    from unsloth import FastLanguageModel
    import torch
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "ehiri/PFE_2024",
        max_seq_length = 1024,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.11: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/2.65G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/3.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [ ]:
def generate_code(description: str) -> str:
    # Tokenize the input description
    inputs = tokenizer(
        [prompt.format(description)],
        return_tensors="pt"
    ).to("cuda")

    # Generate the code without streaming
    output = model.generate(**inputs, max_new_tokens=512)

    # Decode the generated output
    generated_code = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract and return the PySpark code portion
    return generated_code.split("PySpark Code:")[-1].strip()


In [ ]:
prompt = """
Task Description:
{}

PySpark Code:
"""

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("your_Auth_API")

In [ ]:
import nest_asyncio
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from pyngrok import ngrok
import uvicorn


nest_asyncio.apply()

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


class TaskDescription(BaseModel):
    text: str

@app.post("/generate_code/")
async def generate_code_api(task: TaskDescription):
    try:
        generated_code = generate_code(task.text)
        return {"code": generated_code}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/")
async def root():
    return {"message": "PySpark Code Generator API is running"}

In [ ]:
if __name__ == "__main__":
    public_url = ngrok.connect(8000)
    print(f"Public URL: {public_url}")
    uvicorn.run(app, host="0.0.0.0", port=8000)

Public URL: NgrokTunnel: "https://adcc-35-234-55-93.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [705]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     105.158.213.171:0 - "OPTIONS /generate_code/ HTTP/1.1" 200 OK
INFO:     105.158.213.171:0 - "POST /generate_code/ HTTP/1.1" 200 OK
INFO:     105.158.213.171:0 - "POST /generate_code/ HTTP/1.1" 200 OK
